In [1]:
#Importing dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pymongo
import datetime
from flask import Flask
from sqlalchemy import create_engine
import json
#Avoiding setting with copy warning
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Importing Redfin Housing Market State Data
raw_state_df = pd.read_csv("Resources/state_market_tracker.tsv000", sep='\t')
raw_state_df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2019-10-01,2019-10-31,30,state,4,23,f,Oklahoma,NaN,Oklahoma,...,0.098560,0.209877,0.083561,0.053355,0.301282,-0.240768,-0.122115,South Region,NaN,2022-01-09 14:29:56
1,2021-07-01,2021-07-31,30,state,4,40,f,Vermont,NaN,Vermont,...,0.251473,0.133696,0.016402,-0.000131,0.424404,-0.064422,0.076156,Northeast Region,NaN,2022-01-09 14:29:56
2,2016-08-01,2016-08-31,30,state,4,10,f,New Hampshire,NaN,New Hampshire,...,0.089756,0.149826,0.009592,0.002687,0.206101,0.061385,0.016779,Northeast Region,NaN,2022-01-09 14:29:56
3,2013-04-01,2013-04-30,30,state,4,51,f,Mississippi,NaN,Mississippi,...,-0.008566,NaN,NaN,NaN,0.044777,-0.014869,-0.005006,South Region,NaN,2022-01-09 14:29:56
4,2019-12-01,2019-12-31,30,state,4,2,f,Missouri,NaN,Missouri,...,-0.017031,0.205339,-0.123381,-0.000410,0.252082,-0.116423,0.095007,Midwest Region,NaN,2022-01-09 14:29:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27073,2012-03-01,2012-03-31,30,state,4,6,f,Nebraska,NaN,Nebraska,...,0.021426,NaN,NaN,NaN,0.422780,0.042801,0.030384,Midwest Region,NaN,2022-01-09 14:29:56
27074,2016-07-01,2016-07-31,30,state,4,27,f,Idaho,NaN,Idaho,...,0.182390,NaN,NaN,NaN,0.534900,-0.117877,-0.004820,West Region,NaN,2022-01-09 14:29:56
27075,2020-11-01,2020-11-30,30,state,4,42,f,Virginia,NaN,Virginia,...,0.218939,0.126582,-0.048418,-0.006751,0.380567,-0.152381,0.194669,South Region,NaN,2022-01-09 14:29:56
27076,2021-08-01,2021-08-31,30,state,4,40,f,Vermont,NaN,Vermont,...,0.234272,0.158853,0.018284,0.022594,0.381659,-0.018233,0.098021,Northeast Region,NaN,2022-01-09 14:29:56


In [4]:
raw_county_df = pd.read_csv("Resources/county_market_tracker.tsv000", sep='\t')
raw_county_df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2017-04-01,2017-04-30,30,county,5,170,f,"Morgan County, AL",NaN,Alabama,...,-0.069595,NaN,NaN,NaN,0.020833,0.020833,0.001225,"Decatur, AL",19460.0,2022-01-09 14:29:56
1,2014-12-01,2014-12-31,30,county,5,1457,f,"Hennepin County, MN",NaN,Minnesota,...,-0.098136,0.097744,-0.121909,-0.197220,0.172414,0.034483,-0.033469,"Minneapolis, MN",33460.0,2022-01-09 14:29:56
2,2018-02-01,2018-02-28,30,county,5,2319,f,"Tulsa County, OK",NaN,Oklahoma,...,-0.024840,0.206004,-0.026139,0.009313,0.479924,0.068277,-0.042568,"Tulsa, OK",46140.0,2022-01-09 14:29:56
3,2017-08-01,2017-08-31,30,county,5,1448,f,"Crow Wing County, MN",NaN,Minnesota,...,NaN,0.200000,-0.050000,NaN,NaN,NaN,NaN,"Brainerd, MN",14660.0,2022-01-09 14:29:56
4,2019-05-01,2019-05-31,30,county,5,1586,f,"Tate County, MS",NaN,Mississippi,...,0.107895,NaN,NaN,NaN,0.000000,-0.111111,-0.100000,"Memphis, TN",32820.0,2022-01-09 14:29:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563117,2013-03-01,2013-03-31,30,county,5,2728,f,"Gonzales County, TX",NaN,Texas,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,Texas nonmetropolitan area,NaN,2022-01-09 14:29:56
563118,2013-09-01,2013-09-30,30,county,5,710,f,"Valley County, ID",NaN,Idaho,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,Idaho nonmetropolitan area,NaN,2022-01-09 14:29:56
563119,2016-06-01,2016-06-30,30,county,5,1261,f,"Livingston Parish, LA",NaN,Louisiana,...,-0.050180,0.189369,0.010930,0.028419,0.305785,-0.120571,0.020901,"Baton Rouge, LA",12940.0,2022-01-09 14:29:56
563120,2017-09-01,2017-09-30,30,county,5,2972,f,"Gloucester County, VA",NaN,Virginia,...,0.242424,NaN,NaN,NaN,0.000000,-0.111111,-0.166667,"Virginia Beach, VA",47260.0,2022-01-09 14:29:56


In [5]:
raw_us_df = pd.read_csv("Resources/us_national_market_tracker.tsv000", sep='\t')
raw_us_df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2013-09-01,2013-09-30,30,national,11,1400,t,National,National,U.S.,...,0.014642,0.062155,0.003375,0.010655,NaN,NaN,NaN,NaN,NaN,2022-01-09 00:00:00
1,2012-04-01,2012-04-30,30,national,11,1400,f,National,National,U.S.,...,0.018905,0.055948,-0.003248,-0.017449,0.325238,-0.008578,0.062488,NaN,NaN,2022-01-09 14:33:38
2,2018-09-01,2018-09-30,30,national,11,1400,t,National,National,U.S.,...,NaN,0.149476,0.002931,0.030961,NaN,NaN,NaN,NaN,NaN,2022-01-09 00:00:00
3,2019-07-01,2019-07-31,30,national,11,1400,f,National,National,U.S.,...,-0.025856,0.167575,0.006934,0.006512,0.358500,-0.037603,-0.016629,NaN,NaN,2022-01-09 14:33:38
4,2013-01-01,2013-01-31,30,national,11,1400,t,National,National,U.S.,...,0.059250,0.060066,-0.029603,-0.015023,0.351386,-0.010496,0.105331,NaN,NaN,2022-01-09 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,2021-11-01,2021-11-30,30,national,11,1400,f,National,National,U.S.,...,0.123325,0.095114,-0.019921,-0.022888,0.390375,-0.022198,0.069055,NaN,NaN,2022-01-09 14:33:38
1316,2020-03-01,2020-03-31,30,national,11,1400,t,National,National,U.S.,...,NaN,0.130646,-0.005389,-0.019071,NaN,NaN,NaN,NaN,NaN,2022-01-09 00:00:00
1317,2017-05-01,2017-05-31,30,national,11,1400,f,National,National,U.S.,...,0.021272,0.119554,0.013009,-0.003670,0.432571,-0.019318,0.029618,NaN,NaN,2022-01-09 14:33:38
1318,2017-08-01,2017-08-31,30,national,11,1400,f,National,National,U.S.,...,0.031005,0.106648,0.006663,-0.002141,0.383050,-0.002971,0.037232,NaN,NaN,2022-01-09 14:33:38
